In [ ]:
# #legacy code from the original chatbot

# def get_chatGPT_response(query: str, context: List[str]) -> str:
#     """
#     Queries the GPT API to get a response to the question.

#     Args:
#     query (str): The original query.
#     context (List[str]): The context of the query, returned by embedding search.

#     Returns:
#     A response to the question.
#     """
#     response = openai.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=build_prompt(query, context),
#     )

#     return response.choices[0].message.content  # type: ignore


# def build_prompt(query: str, context: List[str]) -> List[ChatCompletionMessageParam]:
#     """
#     Builds a prompt for the LLM. #

#     This function builds a prompt for the LLM. It takes the original query,
#     and the returned context, and asks the model to answer the question based only
#     on what's in the context, not what's in its weights.

#     More information: https://platform.openai.com/docs/guides/chat/introduction

#     Args:
#     query (str): The original query.
#     context (List[str]): The context of the query, returned by embedding search.

#     Returns:
#     A prompt for the LLM (List[ChatCompletionMessageParam]).
#     """

#     system: ChatCompletionMessageParam = {
#         "role": "system",
#         "content": "I am going to ask you a question, which I would like you to answer"
#         "based only on the provided context, and not any other information."
#         "If there is not enough information in the context to answer the question,"
#         'say "I am not sure", then try to make a guess.'
#         "Break your answer up into nicely readable paragraphs.",
#     }
#     user: ChatCompletionMessageParam = {
#         "role": "user",
#         "content": f"The question is {query}. Here is all the context you have:"
#         f'{(" ").join(context)}',
#     }

#     return [system, user]



In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

In [ ]:
for text in llm("AI is going to", stream=True):
    print(text, end="", flush=True)


# results on the cpu using ctransformers
### - 27.4 seconds
### output:
<p>
have an enormous impact on the future of the industry.
This year, AI was all over the show floor and there are a number of companies that are using AI in their products. Here's a quick look at some of them:
Fulcrum is an automated, artificial intelligence-powered software solution that helps manufacturers improve productivity and efficiency by automating processes, removing errors, and freeing up employees to focus on more complex tasks.
Kapil Khera, co-founder and CEO of Fulcrum, told Inman: "I don't know a single company that doesn't have AI in their product. I think it's the most important thing to come out of 2018."
The company is currently working on machine learning tools and workflow applications with machine intelligence technology so you can really begin building these powerful AI-powered solutions. The full AI-focused ecosystem, with automation in product design and sophisticated analytics, will be available to all customers by the end of 2018.
Kapil Khera, co-founder and CEO of Fulcrum
Surely
</p>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [ ]:

def get_response_tiny_llama(text):
    path='/teamspace/studios/this_studio/TinyLlama-1.1B-Chat-v1.0-GPTQ'
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    prompt = text
    system_message = "You are a story writing assistant"
    prompt_template=f'''<|system|>
    {system_message}</s>
    <|user|>
    {prompt}</s>
    <|assistant|>
    '''

    print("\n\n*** Generate:")

    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
    return(tokenizer.decode(output[0]))

In [ ]:
print(get_response_tiny_llama("what is qlora?"))

# result on gpu 2.8 seconds

# handling the embedding model

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)


In [ ]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')

import os
pdf_list=os.listdir("/Users/matansharon/python/opensource_rag/pdf_files")
text_list=os.listdir("/Users/matansharon/python/opensource_rag/text_files")

for pdf in pdf_list:
    temp=pdf.replace(".pdf",".txt")
    if temp not in text_list:
        text=convert_from_pdf_to_text("pdf_files/"+pdf)
        with open("text_files/"+temp, 'w') as f:
            f.write(text)

In [ ]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')


In [ ]:
import chromadb
client=chromadb.Client()
import os
import chromadb.utils.embedding_functions as embedding_functions
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=os.environ.get("HuggingFace_API_KEY"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

collection=client.get_or_create_collection('test',embedding_function=huggingface_ef)


In [ ]:
import chromadb
persist_directory = "chroma_storage"
client = chromadb.PersistentClient(path=persist_directory)
collection=client.get_collection("file1_collection")
print(collection.get().keys())



In [ ]:
results

In [ ]:
results = collection.query(
            query_texts=["summerize the Component Separation method for CMB using Convolutional Neural Networks document"], n_results=5, include=["documents", "metadatas","distances"])
print(results)
for i in range(len(results['distances'][0])):
    if (abs(results['distances'][0][i]-1)<0.4):
        print(results['distances'][0][i])
        print(results['documents'][0][i][:100])
        print(results['metadatas'][0][i])
        print("-"*100)

In [ ]:
#this cell is a testing cell for filtering the results of the query by the distance

temp_results={"ids":[[]],"distances":[[]],"metadatas":[[]],"documents":[[]]}
        for i in range(5):
            if abs(1-results["distances"][0][i]<0.4): 
                temp_results["ids"][0].append(results["ids"][0][i])
                temp_results["distances"][0].append(results["distances"][0][i])
                temp_results["metadatas"][0].append(results["metadatas"][0][i])
                temp_results["documents"].append(results["documents"][0][i])
        results=temp_results

In [ ]:
all_file_names=set()
for line in collection.get()['metadatas']:
    all_file_names.add(line['filename'])
all_file_names

In [ ]:
for doc in collection.get()['documents']:
    print(len(doc))

In [ ]:
import PyPDF2
pages=PyPDF2.PdfReader("/Users/matansharon/python/opensource_rag/pdf_files/qlora.pdf").pages


In [ ]:
# from nltk.tokenize import word_tokenize
# for page in pages:
#     print(len(word_tokenize(page.extract_text())))

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
df=pd.read_csv("salaries.csv")

df_str=df.to_string().strip()
len(word_tokenize(df_str))
with open("salaries2.txt", 'w') as f:
    f.write(df_str)

In [ ]:
all_rows=(df_str.split("\n"))
col_rows_names=all_rows[0]
# print(col_rows_names.strip())
for line in all_rows:
    print((line))

In [ ]:
import anthropic
import os
from dotenv import load_dotenv
load_dotenv()

client=anthropic.Anthropic(
        api_key=os.environ.get("ANTHROPIC_API_KEY")
    )
message = client.messages.create(
        # model="claude-3-opus-20240229",
        model='claude-3-haiku-20240307',
        max_tokens=4000,
        temperature=0.2  ,
        system=f"""
        You are a data analyst assistant.
        your given a dataframe that was converted into string with salaries of employees,
        return the head of the dataframe.
        
        """,
        messages=[
            {"role": "user", "content": df_str}
        ]
    )
print(message.usage)
print( message.content[0].text)



In [ ]:
message

In [ ]:
class Message():
    def __init__(self):
        self.content=''
        self.type=''
    def get_content(self):
        return self.content
    def get_type(self):
        return self.type
        

In [ ]:
import matplotlib.pyplot as plt

# Assuming the dataframe is named 'df'
plt.figure(figsize=(12, 6))
plt.hist(df1000['salary'], bins=20)
plt.xlabel('Salary')
plt.ylabel('Frequency')
plt.title('Salary Distribution')
plt.show()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
# nltk.download('punkt')

files=os.listdir("/Users/matansharon/python/opensource_rag/documents")
files
for file in files:
    if file.endswith(".pdf"):
        print(file)
        pages=convert_from_pdf_to_text("/Users/matansharon/python/opensource_rag/documents/"+file)
        print(len(word_tokenize(pages)))
    elif file.endswith(".csv"):
        df=pd.read_csv("/Users/matansharon/python/opensource_rag/documents/"+file)
        df_str=df.to_string().strip()
        print(df_str)
        # print(len(word_tokenize(df_str)))

In [ ]:
def read_and_convert_csv(file_path: str) -> str:
    df=pd.read_csv(file_path)
    # Convert the dataframe to a string
    df=df.to_string().strip()
    column_row=df.split("\n")[0]
    #split the string into a list of strings
    rows=df.split("\n")
    pages=[]
    page=[]
    for i in range(1,len(rows),10):
        page.append(column_row)
        for j in range(i,i+10):
            if j<len(rows):
            
                page.append(rows[j])
        pages.append(page)
        page=[]
    return pages
res=read_and_convert_csv("/Users/matansharon/python/opensource_rag/documents/salaries.csv")
for page in res:
    for row in page:
        print(row)


In [ ]:
splited=df_str.split('\n')
for row in splited:
    print(row)

In [ ]:


anthropic_client=anthropic.Anthropic(
        api_key=os.environ.get("ANTHROPIC_API_KEY")
    )
context=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/documents/Programming_PyTorch_for_Deep_Learning_Creating_and_Deploying_Deep_Learning_Applications_by_Ian_Pointer_z-lib_org.pdf')
query='summerize this book'
response = anthropic_client.messages.create(
        # model="claude-3-opus-20240229",
        model='claude-3-haiku-20240307',
        max_tokens=1000,
        temperature=0.2,
        system=f"""
        I am going to ask you a question, which I would like you to answer"
        "based only on the provided context, and not any other information."
        "If there is not enough information in the context to answer the question,"
        'say "I am not sure", then try to make a guess.'
        "Break your answer up into nicely readable paragraphs.
        here is all the context you have:{context}"
        """,
        messages=[
            {"role": "user", "content": query}
        ]
    )

In [ ]:
print(response.content[0].text)


In [ ]:
response.usage

In [ ]:
110840/80055

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')
model

In [ ]:
%%time
response = model.generate_content("What is the meaning of life?")

In [ ]:
import chromadb
db_client = chromadb.PersistentClient(path='chroma_storage_pages')
collection=db_client.get_collection('by_pages_collection')
results = collection.query(
            query_texts=['what is qlora?'], n_results=3, include=["documents", "metadatas","distances"],
        )
        
    
context=results["documents"][0]


In [ ]:
context

In [ ]:
import os
import PyPDF2
reader=PyPDF2.PdfReader("/Users/matansharon/python/opensource_rag/pdf_files/Non-linear Welfare-Aware Strategic Learning.pdf")
pages=reader.pages
text=""
for page in pages:
    text+=(page.extract_text())
text

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
len(word_tokenize(text))


In [ ]:
import google.generativeai as genai
import os
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')


response = model.generate_content(f"summerize the following: {text}")
print(response.text)

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import os
import nltk

# nltk.download('punkt')
path="/Users/matansharon/python/opensource_rag/נתוני חברים פברואר - אפריל 2024.xlsx"
df=pd.read_excel(path,sheet_name='גיליון3')
# print(df.to_string())
print("length of tokens of sheet 3: ",len(word_tokenize(df.to_string()))*1.3)




In [ ]:
with open("hours.txt", 'w') as f:
    f.write(df.to_string())


In [ ]:
import anthropic
client=anthropic.Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"))
message = client.messages.create(
        model="claude-3-opus-20240229",
        # model='claude-3-haiku-20240307',
        max_tokens=4000,
        temperature=0.2,
        system=f"""
        You are a data analyst assistant.
        your given a dataframe that was converted into string with employees work hours,
        follow the instructions:
        1. Clean and format the data: Convert the string-based data into a structured format (e.g., a pandas DataFrame) to facilitate data manipulation and analysis.
        2. Explore the data: Perform descriptive statistics, such as calculating the mean, median, and standard deviation for the various work hour categories, to understand the overall distribution and patterns.
        3. Identify key metrics: Determine the most relevant metrics to focus on, such as total work hours, overtime, work from home, and salary, based on the business objectives and priorities.
        4. Analyze trends: Examine the trends in the key metrics over time, looking for any significant changes or patterns that may indicate areas for improvement or further investigation.
        5. Identify outliers and anomalies: Detect any unusual or unexpected values in the data, which could represent potential issues or areas for further exploration.
        6. Correlate data points: Investigate the relationships between different data points, such as work hours and salary, to uncover any insights or potential dependencies.
        7. Visualize the data: Create informative visualizations, such as line charts, bar graphs, or heatmaps, to effectively communicate the findings and trends to stakeholders.
        
        
        """,
        messages=[
            {"role": "user", "content": df.to_string()}
        ]
    )

In [ ]:
print(message.content[0].text)

In [ ]:
with open('output.txt', 'w') as f:
    f.write(message.content[0].text)

In [ ]:
import matplotlib.pyplot as plt

attendance_hours = [14399.46, 2473.46, 1775.5, 739.43, 94.5, 1476.57, 3144]
attendance_labels = ['Present for Salary', 'Vacation', 'Work from Home', 'Sick Leave', 'Military Service', 'Security Absence (Code 62)', 'Absence']

plt.figure(figsize=(12, 6))
plt.pie(attendance_hours, labels=attendance_labels, autopct='%1.1f%%')
plt.title('Attendance and Absence Breakdown')
plt.axis('equal')
plt.show()

In [ ]:
import os
#generate a code that walks through the directory and all the subdirectories and return a list of all files
def get_all_files(path):
    files=[]
    for root, dirs, filenames in os.walk(path):
        for filename in filenames:
            files.append(os.path.join(root,filename))
    return files
def read_pdf(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
#gereate a code that read .doc or .docx files and return the text
# def read_docx(file_path):
#     from docx import Document
#     doc = Document(file_path)
#     res=''
#     for para in doc.paragraphs:
#         res+=para.text
#     return res
# from spire.doc import *
# from spire.doc.common import *
        
# # Create a Document object
# document = Document()
# # Load a Word DOCX file
# document.LoadFromFile("Sample.docx")
# # Or load a Word DOC file
# #document.LoadFromFile("Sample.doc")

# # Save the file to a PDF file
# document.SaveToFile("WordToPdf.pdf", FileFormat.PDF)
# document.Close()



In [ ]:
# files=get_all_files("/Users/matansharon/python/opensource_rag/2020")
# from docx2pdf import convert
# pdf_cnt=0
# docx_cnt=0
# doc_cnt=0
# for file in files:
#     if file.endwith(".docx"):
#         docx_cnt+=1
        
#         new_path=file.replace(".docx",".pdf")
        
    
# print("pdf:",pdf_cnt,"docx:",docx_cnt,"doc:",doc_cnt)
# # read_pdf('/Users/matansharon/python/opensource_rag/2020/ECO-20-0082-שינוי תהליך טיפול בשינויים הנדסיים/ECO-20-0082-שינוי תהליך טיפול בשינויים הנדסיים.pdf')


In [ ]:
for file in files:
    if file.endswith(".pdf"):
        print("pdf->"*10)
        print(read_pdf(file))



In [65]:
import os
import chromadb
db=chromadb.PersistentClient(path='chroma_storage_pages')
collection=db.get_collection('by_pages_collection')
query='test_train_split'

results = collection.query(
            query_texts=[query], n_results=10, include=["documents", "metadatas","distances"],
        )
# import numpy as np
#filter the results by the distance the criteria is that the distance should be by this formula: abs(1-distance)<0.4   

def filter_results(data):
    # Unpack data
    ids = data.get('ids', [])
    distances = data.get('distances', [])
    metadatas = data.get('metadatas', [])
    documents = data.get('documents', [])
    
    filtered_results = {'ids': [], 'distances': [], 'metadatas': [], 'documents': []}
    
    # Iterate over distances and filter based on the criteria
    for i, distance_list in enumerate(distances):
        for j, distance in enumerate(distance_list):
            if abs(1 - distance) < 0.4:
                filtered_results['ids'].append(ids[i][j])
                filtered_results['distances'].append(distance)
                filtered_results['metadatas'].append(metadatas[i][j])
                filtered_results['documents'].append(documents[i][j])
                
    return filtered_results

# Example usage


filter_results(results)



{'ids': ['1186',
  '1139',
  '1175',
  '1220',
  '1023',
  '1153',
  '1104',
  '1113',
  '1088',
  '1221'],
 'distances': [1.2508236169815063,
  1.254045009613037,
  1.2796638011932373,
  1.2918291113339946,
  1.293965458869934,
  1.3380935192108154,
  1.3382604122161865,
  1.34063720703125,
  1.3589706420898438,
  1.3597797106378184],
 'metadatas': [{'filename': 'Programming_PyTorch_for_Deep_Learning_Creating_and_Deploying_Deep_Learning_Applications_by_Ian_Pointer_z-lib_org.pdf',
   'page_number': 147},
  {'filename': 'Programming_PyTorch_for_Deep_Learning_Creating_and_Deploying_Deep_Learning_Applications_by_Ian_Pointer_z-lib_org.pdf',
   'page_number': 99},
  {'filename': 'Programming_PyTorch_for_Deep_Learning_Creating_and_Deploying_Deep_Learning_Applications_by_Ian_Pointer_z-lib_org.pdf',
   'page_number': 136},
  {'filename': 'Programming_PyTorch_for_Deep_Learning_Creating_and_Deploying_Deep_Learning_Applications_by_Ian_Pointer_z-lib_org.pdf',
   'page_number': 181},
  {'filename':